<img src="res/th-logo.png" width="100" align="left"/>
<img src="res/th-bar.png" width="800" align="left"/>

<h1 align="center"> WPF Cyber-physische Systeme (CPS) </h1><br>
<h2 align="center"> Niryo NED2 </h2><br><br>

<img src="res/Niryo_logo.png" width="300"/><br><br>
<img src="res/titelbild.png" width="500"/><br><br><br>


<h4 align="center"> System </center><br>
<img src="res/ros_logo.png" width="100"/><br><br>

Der Roboterarm NED2 läuft unter der Middleware **ROS**.



<h4 align="center"> NiryoStudio </h4><br>
Kontrolliert werden kann der NED2 über das NiryoStudio. Über dieses Programm muss der Roboter immer mit dem Rechner verbunden werden, damit der Code auf dem Roboter ausgeführt werden kann. In Sciebo liegt die Anwendung im Pfad: WPF_CPS_SoSe23\NED2. Alternativ kann diese unter dem folgenden Link heruntergeladen werden (dazu müssen Sie sich allerdings bei Niryo registrieren und benötigen die Seriennr. unseres Roboterarms):
https://docs.niryo.com/product/niryo-studio/v4.1.1/en/source/download_and_installation.html

<h4 align="center">  Programmierung </h4><br>
<img src="res/PyNiryo_logo_2.png" width="200"/><br><br>

Programmiert werden kann der NED2 in jeder Entwicklungsumgebung, die sich auf dem durch NiryoStudio verbundenen Rechner befindet. Der Code kann hier in den Notebooks ausgeführt werden.

In diesem WPF wird Python mit dem Package **PyNiryo** genutzt, um mit dem NED2 zu interagieren. Im Folgenden werden Sie ein bereits vorbereitetes Virtual Environment aktivieren, wenn Sie dieses Notebook nicht aus einer Anaconda Umgebung heraus gestartet haben.

## Projekt 1: Fertigungsprozess mit Niryo NED2
Die folgenden Aufgabenblöcke können unter den Teams mit jeweils einem Thema aufgeteilt werden. Es sollen verschiedene Aufgaben zur Implementierung eines industriellen Fertigungsprozesses gelöst werden.

### NED2 mit Vision-Funktion und Förderband: Transfer gefertigter Objekte zur Versandabteilung
Sie sind für die Programmierung eines Roboter-Arms einer Fertigungsstraße zuständig. Gefertigte Objekte werden in ein Zwischenlager gebracht, bevor sie auf Fehler überprüft und zum Transport in die Versandabteilung auf ein Förderband gelegt werden.

#### Workspace einrichten
Die erste Aufgabe besteht darin, das Zwischenlager als Workspace in NiryoStudio einzurichten. Folgen Sie dazu den Instruktionen unter dem Reiter "Vision Module".
Positionieren Sie das Feld mit den runden Markierungen vor dem NED2. Platzieren Sie mehrere Objekte in der Mitte des Felds. Erstellen Sie in NiryoStudio einen neuen Workspace Warenlager. Beachten Sie dabei die Instruktionen des Programms, insbesondere das Lösen des Greifers und das Platzieren des Kalibrierungsmoduls für Workspaces. Schalten Sie den NED2 vor den Montierungsarbeiten unbedingt aus, um Schäden durch ungewollte Bewegungen oder Kontakt mit unter elektrischer Spannung stehenden Anschlüssen zu vermeiden! 

<img src="res/niryostudio_workspace_zwischenlager.png" width="600" align="left"/>

#### Konfiguration der Arbeitsumgebung
Dieser Schritt wird nur benötigt, wenn Sie nicht mit Anaconda arbeiten. 

Führen Sie nun den nachfolgenden Code aus, um Ihr System zur Interaktion mit dem NED2 zu konfigurieren. Das Venv wurde bereits erzeugt, normalerweise müssen die auskommentierten Anweisungen nicht mehr ausgeführt werden! Bei Ausführung des folgenden Blocks sollte das Venv "niryo_venv" als aktive Umgebung angezeigt werden, das alle nötigen Module enthält, die für die folgenden Aufgaben benötigt werden.

In [1]:
# Virtual Environment installieren (muss nur einmal gemacht werden)
#! python3 -m venv niryo_venv
#! pip3 install pyniryo

# Virtual Environment einrichten
#! source niryo_venv/bin/activate
#! pip3 -V

#### Grundeinstellungen des NED2
Der folgende Block legt die Grundeinstellungen des NED2 fest. Normalerweise muss hier nichts mehr verändert werden (mit simulation_mode bestimmen Sie, ob Sie mit der Simulation oder mit dem realen Roboter arbeiten. Im Simulationsmodus müssen Sie vermutlich auch die robot_ip_address anpassen). Führen Sie die folgenden beiden Blöcke aus, um mit dem NED2 interagieren zu können. Sie werden eine visuelle und akustische Rückmeldung vom NED2 erhalten (in der Simulation nicht).

In [8]:
# https://docs.niryo.com/dev/pyniryo2/v1.0.0/en/index.html
from pyniryo2 import *

# Sie werden auch für bestimmte Funktionen pyniryo brauchen. Zum Beispiel uncompress_image, undistort_image.
# https://docs.niryo.com/dev/pyniryo/v1.1.2/en/index.html
import pyniryo


# Netzwerk
hotspot_mode = "10.10.10.10"
wifi_mode = "192.168.0.140"

# -- MUST Change these variables
simulation_mode = False
if simulation_mode:
    robot_ip_address, workspace_name = "192.168.209.128", "gazebo_1"
else:
    robot_ip_address, workspace_name = wifi_mode, "cps_praktikum"
    

# -- Can Change these variables
grid_dimension = (3, 3)  # conditioning grid dimension
vision_process_on_robot = False  # boolean to indicate if the image processing happens on the Robot
display_stream = True  # Only used if vision on computer

# -- Should Change these variables
# The pose from where the image processing happens
observation_pose = PoseObject(
    x=0.17, y=0., z=0.35,
    roll=0.0, pitch=1.57, yaw=0.0,
)

# Center of the conditioning area
center_conditioning_pose = PoseObject(
    x=0.0, y=-0.25, z=0.12,
    roll=-0., pitch=1.57, yaw=-1.57
)

In [9]:
# Connect to robot
robot_ip_address = '192.168.187.128'
robot = NiryoRobot(robot_ip_address)

if (not simulation_mode):
    # Calibrate robot if robot needs calibration
    robot.arm.calibrate_auto()
    # Equip tool
    robot.tool.update_tool()

# Launching main process
# Der Roboter braucht eine Verbindung zum Internet für robot.sound.say
#ROBOT.SOUND.SAY FUNKTIONIERT NICHT: "Message : Wrong robot hardware version, feature only available on Ned2")
#Annahme, funktioniert nicht in der Simulation
#robot.sound.say("Configuration successful", 0)
# Ending
robot.arm.go_to_sleep()
# Releasing connection
#robot.end()

#### Objekte greifen und positionieren
Programmieren Sie eine Funktion, die Objekte aufgrund von visuellen Merkmalen (Farbe, Form) voneinander unterscheiden kann. Die gefertigten Objekttypen sind vorher bekannt. Es sollte eine entsprechende Rückmeldung von NED2 erfolgen, wenn ein Objekt erkannt wurde (z.B. Markierung im Bild).

<img src="res/python_image_tresh_any.png" width="600" align="left"/>

Sobald die Objekte sicher erkannt werden können, können diese vom Zwischenlager auf das Förderband in Richtung Verpackungsprozess gelegt werden. Erweitern Sie die Funktion aus der letzten Aufgabe, sodass alle Objekte im Lager nacheinander gegriffen und über der Rampe losgelassen werden, damit diese auf das Förderband gelangen.

Testen Sie Ihren Code zunächst in der Simulation bevor Sie diesen am realen Roboter testen. In der Simulation gibt es keine Rampe. In der Simulation können die Objekte deshalb an irgendeinem Ort abgelegt werden. 

#### Hauptprogramm

In [49]:
#platziert alle Objekte an einen Ort
offset_size = 0.05
workspace_name = 'gazebo_1'
# The pick pose
pick_pose = PoseObject(
    x=0.25, y=0., z=0.15,
    roll=-0.0, pitch=1.57, yaw=0.0,
)
# The Place pose
place_pose = PoseObject(
    x=0.0, y=-0.25, z=0.1,
    roll=0.0, pitch=1.57, yaw=-1.57)

delete_pose= PoseObject(
    x=0.225, y=0.32, z=0.08,
    roll=-0.0, pitch=1.4, yaw=0.8)

robot.arm.move_pose(observation_pose)

#robot.arm.set_learning_mode(True)
obj_found, shape_ret, color_ret = robot.vision.vision_pick(workspace_name)
catch_count = 0
while obj_found:
    next_place_pose = place_pose.copy_with_offsets(x_offset= catch_count * offset_size)
    robot.pick_place.place_from_pose(next_place_pose)
    robot.tool.open_gripper()
    robot.arm.move_pose(observation_pose)
    obj_found, shape_ret, color_ret = robot.vision.vision_pick(workspace_name)
    if obj_found == False:
        break


ObjectColor.BLUE
ObjectColor.RED


#### Optional: Fehlerhafte Objekte aussortieren
Leider ist der Produktionsabteilung ein Fehler unterlaufen: Es wurden keine roten und keine blauen, runden Objekte bestellt. Identifizieren Sie Objekte, die diese Eigenschaften erfüllen, um diese nachträglich vom Förderband zu entfernen. Dies muss allerdings spätestens bis zur zweiten Markierung geschehen, da die Objekte sonst in die Versandabteilung gelangen. NED2 sollte beide Aufgaben abwechselnd erledigen. In der Praxis könnten hier zwei verschiedene Roboter-Arme zum Einsatz kommen. NED2 könnte hier bspw. drei Objekte aus dem Lager greifen und platzieren, bevor er an anderer Stelle des Förderbandes kontrolliert, ob es fehlerhafte Objekte gibt, und diese ggf. greift und entfernt. In diesem Falle sollte das Band so lange angehalten werden, bis das unerwünschte Objekt entfernt wurde.

<img src="res/python_image_tresh_red.png" width="600" align="left"/>

In [52]:
#Separiert runde, blau/rote Objekte von allen restlichen
#Letzlich landen alle im selben Top, weil es keine runden Objekte gibt

#WELCHEN TYP HABEN RUNDE OBJEKTE?

offset_size = 0.05
workspace_name = 'gazebo_1'
# The pick pose
pick_pose = PoseObject(
    x=0.25, y=0., z=0.15,
    roll=-0.0, pitch=1.57, yaw=0.0,
)
# The Place pose
place_pose = PoseObject(
    x=0.0, y=-0.25, z=0.1,
    roll=0.0, pitch=1.57, yaw=-1.57)

delete_pose= PoseObject(
    x=0.225, y=0.32, z=0.08,
    roll=-0.0, pitch=1.4, yaw=0.8)

robot.arm.move_pose(observation_pose)

#robot.arm.set_learning_mode(True)
obj_found, shape_ret, color_ret = robot.vision.vision_pick(workspace_name)
catch_count = 0
delete_count= 0
while obj_found:
    next_place_pose = place_pose.copy_with_offsets(x_offset= catch_count * offset_size)
    next_delete_pose= place_pose.copy_with_offsets(x_offset= catch_count * offset_size)
    #Blau wird  gesondert platziert
    if shape_ret is ObjectShape.CIRCLE and (color_ret is ObjectColor.BLUE or color_ret is ObjectColor.RED):
        robot.pick_place.place_from_pose(delete_pose)
        robot.tool.open_gripper()
        robot.arm.move_pose(observation_pose)
        delete_count += 1
    #alle anderen Farben kommen auf place_pose
    else:
        robot.pick_place.place_from_pose(next_place_pose)
        #zusätzlich hinzugefügt, weil er sonst nicht loslässt (Ursache unbekannt)
        robot.tool.open_gripper()
        robot.arm.move_pose(observation_pose)
        catch_count +=1
    obj_found, shape_ret, color_ret = robot.vision.vision_pick(workspace_name)
    if obj_found == False:
        break


In [ ]:
#Die Objkete einfach trennen im Vergleich zum vorherigen Code

# -- Ihr Code hier --
#definiere eine Bestellliste
#Laufband läuft solange bis ein Objekt die Lichtschranke erreicht
# Platziere jedes Objekt vom Lager auf das Förderband
# Wenn ein Objekt die Lichtschranke erreicht:
# überprüfe Objekt:
# wenn richtig, lass Förderband laufen und wenn es noch Objekte im Lager gibt, mache weiter, sonst überwache weiter die Lichtschranke.
# wenn falsch, sortiere das Objekt aus und [...]

#### Optional: Protokoll führen
Erweitern Sie die Funktion um eine Protokollführung. Für die Bilanz des Unternehmens sollen die Objekte in einer Logdatei erfasst werden, die erfolgreich zur Versandabteilung geschickt wurden, sowie jene, die aufgrund von Produktionsfehlern aussortiert werden mussten.

In [57]:
# -- Ihr Code hier --

protokoll = []

def protokolliere(shape, color, correct):
    protokoll.append((shape, color, correct))
#Separiert runde, blau/rote Objekte von allen restlichen
#Letzlich landen alle im selben Top, weil es keine runden Objekte gibt

#WELCHEN TYP HABEN RUNDE OBJEKTE?

offset_size = 0.05
workspace_name = 'gazebo_1'
# The pick pose
pick_pose = PoseObject(
    x=0.25, y=0., z=0.15,
    roll=-0.0, pitch=1.57, yaw=0.0,
)
# The Place pose
place_pose = PoseObject(
    x=0.0, y=-0.25, z=0.1,
    roll=0.0, pitch=1.57, yaw=-1.57)

delete_pose= PoseObject(
    x=0.225, y=0.32, z=0.08,
    roll=-0.0, pitch=1.4, yaw=0.8)

robot.arm.move_pose(observation_pose)

#robot.arm.set_learning_mode(True)
obj_found, shape_ret, color_ret = robot.vision.vision_pick(workspace_name)
catch_count = 0
delete_count= 0
while obj_found:
    next_place_pose = place_pose.copy_with_offsets(x_offset= catch_count * offset_size)
    next_delete_pose= place_pose.copy_with_offsets(x_offset= catch_count * offset_size)
    #Blau wird  gesondert platziert
    if shape_ret is ObjectShape.CIRCLE and (color_ret is ObjectColor.BLUE or color_ret is ObjectColor.RED):
        robot.pick_place.place_from_pose(delete_pose)
        robot.tool.open_gripper()
        robot.arm.move_pose(observation_pose)
        delete_count += 1
        protokolliere((shape_ret, color_ret, False))
    #alle anderen Farben kommen auf place_pose
    else:
        robot.pick_place.place_from_pose(next_place_pose)
        #zusätzlich hinzugefügt, weil er sonst nicht loslässt (Ursache unbekannt)
        robot.tool.open_gripper()
        robot.arm.move_pose(observation_pose)
        catch_count +=1
        protokolliere(shape_ret, color_ret, True)
    obj_found, shape_ret, color_ret = robot.vision.vision_pick(workspace_name)
    if obj_found == False:
        break
print(protokoll)

[(<ObjectShape.SQUARE: 'SQUARE'>, <ObjectColor.GREEN: 'GREEN'>, True), (<ObjectShape.SQUARE: 'SQUARE'>, <ObjectColor.BLUE: 'BLUE'>, True), (<ObjectShape.SQUARE: 'SQUARE'>, <ObjectColor.RED: 'RED'>, True)]


#### Optional: Den NED2 gesprächig machen
Der NED2 kann sowohl visuelle als auch akustische Signale ausgeben. Der LED-Ring des NED2 kann in RGB-Farben eingestellt werden. Akustische Signale können in Tönen oder Sprache erfolgen.

Fügen Sie folgende Signale hinzu, damit der NED2 über den gesamten Arbeitsverlauf kommunikativer ist:

- LED-Ring soll die Farbe des gerade fokussierten Objekts annehmen, bis das Objekt gegriffen wurde. Dazu soll er dann den Satz sagen: "*objektfarbe* Objekt erkannt" (benötigt Internetverbindung des Roboters - aktuell nicht möglich).